In [5]:
import pandas as pd
import httpx
import pybaseball

In [23]:
def fetch_mlb_standings():
    url = "https://statsapi.mlb.com/api/v1/standings?leagueId=103,104&season=2025&standingsTypes=regularSeason"
    response = httpx.get(url)
    response.raise_for_status()
    return response.json()

In [24]:
data = fetch_mlb_standings()

In [25]:
type(data)

dict

In [26]:
data

{'copyright': 'Copyright 2025 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'records': [{'standingsType': 'regularSeason',
   'league': {'id': 103, 'link': '/api/v1/league/103'},
   'division': {'id': 201, 'link': '/api/v1/divisions/201'},
   'sport': {'id': 1, 'link': '/api/v1/sports/1'},
   'lastUpdated': '2025-04-25T04:52:16.232Z',
   'teamRecords': [{'team': {'id': 147,
      'name': 'New York Yankees',
      'link': '/api/v1/teams/147'},
     'season': '2025',
     'streak': {'streakType': 'wins', 'streakNumber': 1, 'streakCode': 'W1'},
     'divisionRank': '1',
     'leagueRank': '2',
     'sportRank': '7',
     'gamesPlayed': 25,
     'gamesBack': '-',
     'wildCardGamesBack': '-',
     'leagueGamesBack': '-',
     'springLeagueGamesBack': '-',
     'sportGamesBack': '3.0',
     'divisionGamesBack': '-',
     'conferenceGamesBack': '-',
     'leagueRecord': {'wins': 15, 'l

In [27]:
mil_brew = data["records"][4]['teamRecords'][0]

In [133]:
def parse_standings(raw_data):
    teams = []
    for div in raw_data["records"]:
        division_id = div['division']['id']
        for team_entry in div['teamRecords']:
            team = team_entry['team']['name']
            team_id = team_entry['team']['id']
            wins = team_entry['leagueRecord']['wins']
            losses = team_entry['leagueRecord']['losses']
            win_percentage = team_entry['leagueRecord']['pct']
            games_back = team_entry['gamesBack']
            division_rank = team_entry['divisionRank']
            league_rank = team_entry['leagueRank']
            mlb_rank = team_entry['sportRank']
            games_played = team_entry['gamesPlayed']
            teams.append({
                'team': team,
                'team_id': team_id,
                'wins': wins,
                'losses': losses,
                'win_percentage': win_percentage,
                'games_back': games_back,
                'division_id': division_id,
                'division_rank': division_rank,
                'league_rank': league_rank,
                'mlb_rank': mlb_rank,
                'games_played': games_played
            })
    return pd.DataFrame(teams)

In [132]:
# data['records'][2]['teamRecords'][0]['leagueRecord']['pct']
# data['records'][2]['teamRecords'][0]['sportRank']
data['records'][2]['teamRecords'][0]['team']['id']

136

In [134]:
parse_standings(data)


,team,team_id,wins,losses,win_percentage,games_back,division_id,division_rank,league_rank,mlb_rank,games_played
0,New York Yankees,147,15,10,.600,-,201,1,2,7,25
1,Boston Red Sox,111,14,13,.519,2.0,201,2,7,14,27
2,Toronto Blue Jays,141,12,13,.480,3.0,201,3,10,19,25
3,Tampa Bay Rays,139,11,14,.440,4.0,201,4,12,22,25
4,Baltimore Orioles,110,10,14,.417,4.5,201,5,13,25,24
5,Detroit Tigers,116,15,10,.600,-,202,1,1,6,25
6,Cleveland Guardians,114,14,10,.583,0.5,202,2,3,8,24
7,Kansas City Royals,118,12,14,.462,3.5,202,3,11,20,26
8,Minnesota Twins,142,9,16,.360,6.0,202,4,14,28,25
9,Chicago White Sox,145,6,19,.240,9.0,202,5,15,29,25


In [137]:
team_temp = parse_standings(data)
type(team_temp)

pandas.core.frame.DataFrame

In [184]:
def get_div_leader(df):
    div_leader = df.loc[df.groupby('division_id')['division_rank'].idxmin()]
    return div_leader.reset_index(drop=True)

get_div_leader(team_temp)

,team,team_id,wins,losses,win_percentage,games_back,division_id,division_rank,league_rank,mlb_rank,games_played
0,Seattle Mariners,136,14,11,.560,-,200,1,4,10,25
1,New York Yankees,147,15,10,.600,-,201,1,2,7,25
2,Detroit Tigers,116,15,10,.600,-,202,1,1,6,25
3,San Diego Padres,135,17,8,.680,-,203,1,2,2,25
4,New York Mets,121,18,7,.720,-,204,1,1,1,25
5,Chicago Cubs,112,16,10,.615,-,205,1,5,5,26


In [218]:
def cross_division_ranks(df):
    divisions = df['division_id'].astype(str).unique()
    division_leaders = get_div_leader(df)
    division_leaders['divsion_id'] = division_leaders['division_id'].astype(str)
    for i in divisions:
        df['div_id_' + i] = ""

    for index, row in df.iterrows():
        for _, leader_row in division_leaders.iterrows():
            if row['division_id'] != leader_row['division_id']:
                if int(row['mlb_rank']) < int(leader_row['mlb_rank']):
                    df.at[index, 'div_id_' + str(leader_row['division_id'])] = True
                else:
                    df.at[index, 'div_id_' + str(leader_row['division_id'])] = False

    return df
cross_division_ranks(team_temp)

,team,team_id,wins,losses,win_percentage,games_back,division_id,division_rank,league_rank,mlb_rank,games_played,div_id_201,div_id_202,div_id_200,div_id_204,div_id_205,div_id_203
0,New York Yankees,147,15,10,.600,-,201,1,2,7,25,,False,True,False,False,False
1,Boston Red Sox,111,14,13,.519,2.0,201,2,7,14,27,,False,False,False,False,False
2,Toronto Blue Jays,141,12,13,.480,3.0,201,3,10,19,25,,False,False,False,False,False
3,Tampa Bay Rays,139,11,14,.440,4.0,201,4,12,22,25,,False,False,False,False,False
4,Baltimore Orioles,110,10,14,.417,4.5,201,5,13,25,24,,False,False,False,False,False
5,Detroit Tigers,116,15,10,.600,-,202,1,1,6,25,True,,True,False,False,False
6,Cleveland Guardians,114,14,10,.583,0.5,202,2,3,8,24,False,,True,False,False,False
7,Kansas City Royals,118,12,14,.462,3.5,202,3,11,20,26,False,,False,False,False,False
8,Minnesota Twins,142,9,16,.360,6.0,202,4,14,28,25,False,,False,False,False,False
9,Chicago White Sox,145,6,19,.240,9.0,202,5,15,29,25,False,,False,False,False,False


In [ ]:
team_temp['division_rank'].rank()

In [201]:
divisions = team_temp['division_id'].unique()
divisions

array([201, 202, 200, 204, 205, 203])

In [182]:
tmp_div = team_temp[team_temp['division_id'] == 200]
tmp_div['division_rank'].min()


'1'

In [154]:
team_temp

,team,team_id,wins,losses,win_percentage,games_back,division_id,division_rank,league_rank,mlb_rank,games_played
0,New York Yankees,147,15,10,.600,-,201,1,2,7,25
1,Boston Red Sox,111,14,13,.519,2.0,201,2,7,14,27
2,Toronto Blue Jays,141,12,13,.480,3.0,201,3,10,19,25
3,Tampa Bay Rays,139,11,14,.440,4.0,201,4,12,22,25
4,Baltimore Orioles,110,10,14,.417,4.5,201,5,13,25,24
5,Detroit Tigers,116,15,10,.600,-,202,1,1,6,25
6,Cleveland Guardians,114,14,10,.583,0.5,202,2,3,8,24
7,Kansas City Royals,118,12,14,.462,3.5,202,3,11,20,26
8,Minnesota Twins,142,9,16,.360,6.0,202,4,14,28,25
9,Chicago White Sox,145,6,19,.240,9.0,202,5,15,29,25


In [155]:
team_temp[team_temp['games_back'] == '-']

,team,team_id,wins,losses,win_percentage,games_back,division_id,division_rank,league_rank,mlb_rank,games_played
0,New York Yankees,147,15,10,.600,-,201,1,2,7,25
5,Detroit Tigers,116,15,10,.600,-,202,1,1,6,25
10,Seattle Mariners,136,14,11,.560,-,200,1,4,10,25
11,Texas Rangers,140,14,11,.560,-,200,2,5,11,25
15,New York Mets,121,18,7,.720,-,204,1,1,1,25
20,Chicago Cubs,112,16,10,.615,-,205,1,5,5,26
25,San Diego Padres,135,17,8,.680,-,203,1,2,2,25
